# Visit individual Calexp from Selected visits

 work with Weakly_2024_10
- use jupyter kernel LSST
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/03/19
- update : 2024/03/19

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [ ]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom

In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

Import statements that we will need later

Let's make a new plot and metric tool, we'll base it on the example in the getting started guide.

In [ ]:
skymap_sel = "latiss_v1"
instrument_sel = "LATISS"
tract_sel = 3864
patch_sel = 236 # or 236
band_sel = "g"


# plotting range for visits
XLIM = (122.3,122.7)
YLIM = (-36.3,-36.05)

In [ ]:
#dataId = {"skymap": "latiss_v1", "tract": 5615, "instrument": "LATISS"}
dataId = {"skymap": skymap_sel, "tract": tract_sel, "instrument": instrument_sel}
#collections="LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
collections = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"
repo = '/sdf/group/rubin/repo/oga/'
butler = dafButler.Butler(repo)
#t = Butler.get(table_sel, dataId=dataId, collections=collections)
registry = butler.registry

In [ ]:
skymap_auxtel ='latiss_v1'
skymap = butler.get('skyMap', skymap=skymap_auxtel, collections=collections)

In [ ]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

## Selected visits

In [ ]:
file_selected_visits = "../data/202402/SelectedVisits_fall2023_tract_3864.csv"

df_myselectedvisits = pd.read_csv(file_selected_visits,index_col=0)

# select the band
cut = df_myselectedvisits.band == band_sel 
df_myselectedvisits = df_myselectedvisits[cut]

my_selectedvisits = list(df_myselectedvisits.index)

#move the visitid as a column not to loose it during the merge 
df_myselectedvisits.reset_index(inplace=True) 

In [ ]:
len(my_selectedvisits) 

In [ ]:
index=10
visitId = my_selectedvisits[index]

In [ ]:
datasetType = 'calexp'
dataId = {'visit': visitId, 'instrument':instrument_sel , 'detector': 0}
datasetRefs = registry.queryDatasets(datasetType, dataId=dataId, collections  =collections)

for i, ref in enumerate(datasetRefs):
    print(ref.dataId)
    print("band:", ref.dataId['band'])
    band = ref.dataId['band']

## Sources in Calexp

In [ ]:
# Create a basic schema to use with these tasks
schema = afwTable.SourceTable.makeMinimalSchema()
print(schema)

In [ ]:
tab = afwTable.SourceTable.make(schema)

In [ ]:
src_cat = butler.get('src', dataId= dataId, collections=collections)

In [ ]:
schema = src_cat.getSchema()
tab = afwTable.SourceTable.make(schema)

In [ ]:
src_cat.asAstropy().to_pandas()

## Calexp

In [ ]:
%matplotlib widget
img = butler.get("calexp", dataId=dataId, collections=collections)

wcs = img.getWcs()
bbox = img.getBBox()
corners_ra, corners_dec = get_corners_radec(wcs, bbox)
extent = [np.min(corners_ra),np.max(corners_ra),np.min(corners_dec),np.max(corners_dec)]


# Display the image with a suitable scaling
med = np.median(img.image.array.flatten())
sigMad = 1.4826 * np.median(np.fabs(img.image.array.flatten() - med))
vmin = med - 3 * sigMad
vmax = med + 3 * sigMad
fig,ax = plt.subplots(1,1,figsize=(10,10))
im=ax.imshow(img.image.array, origin="lower",vmin=vmin, vmax=vmax, cmap="Greys",extent=extent)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
the_title = f"Calexp : visit = {visitId}, band = {band}"
ax.set_title(the_title)
ax.grid()
plt.colorbar(im, cax=cax)
plt.suptitle(collections)
plt.tight_layout()
plt.show()

In [ ]:
calexp_metadata = dict(img.getMetadata())

In [ ]:
calexp_metadata_keys= list(calexp_metadata.keys())
SELECTED_KEYS = ['CCDGAIN', 'CCDNOISE', 'LSST_NUM', 'FILTBAND','FILTER' , 'DATE', 'MJD', 'IMGTYPE', 'DATE-TRG', 'MJD-TRG', 'OBSID', 
                 'DATE-BEG', 'MJD-BEG', 'DATE-END', 'MJD-END', 'GROUPID', 'RA', 'DEC', 'RASTART', 'DECSTART', 'RAEND', 'DECEND', 'ROTPA', 'ROTCOORD', 'HASTART', 'ELSTART', 
                 'AZSTART', 'AMSTART','DAYOBS', 'SEQNUM',  'BGMEAN', 'BGVAR', 'SFM_ASTROM_OFFSET_MEAN', 'SFM_ASTROM_OFFSET_STD', 'MAGZERO', 'MAGZERO_RMS', 'MAGZERO_NOBJ']
for key in calexp_metadata_keys:
    if key in SELECTED_KEYS:
        print(key,calexp_metadata[key])

In [ ]:
wcs = img.getWcs()
bbox = img.getBBox()
img.getPhotoCalib()

In [ ]:
get_corners_radec(wcs, bbox)

In [ ]:
fitsMd = wcs.getFitsMetadata()
WCSfMd = WCS(fitsMd)

In [ ]:
%matplotlib widget    
# Plot the calexp we just retrieved
plt.figure(figsize=(10,10))
plt.subplot(projection= WCSfMd)
    
transform = AsinhStretch() + ZScaleInterval()
scaledImage = transform(img.image.array)
    #scaledImage = np.flipud(scaledImage)
plt.imshow(scaledImage,origin="lower",aspect='equal',cmap="gray")
plt.suptitle(the_title,fontsize=10)

ax = plt.gca()
overlay = ax.get_coords_overlay('icrs')
overlay.grid(color='white', ls='dotted')
#plt.tight_layout()
plt.show()


## Footprints

https://pipelines.lsst.io/modules/lsst.afw.detection/Footprint.html

In [ ]:
from lsst.meas.algorithms.detection import SourceDetectionTask
sourceDetectionTask = SourceDetectionTask(schema=schema)
result = sourceDetectionTask.run(tab, img)

In [ ]:
# Grab the above-threshold footprints that were detected,
#  and assign them to the variable `fps`
fpset = result.positive
fps = fpset.getFootprints()

In [ ]:
len(fps)

In [ ]:
#initial_psf_stars_footprints = butler.get("initial_stars_footprints_detector", dataId=dataId, collections=collections)
#[DatasetType('initial_psf_stars_footprints', {band, instrument, detector, physical_filter, visit}, SourceCatalog),
# DatasetType('initial_stars_footprints_detector', {band, instrument, detector, physical_filter, visit}, SourceCatalog),
# DatasetType('initial_psf_stars_footprints_detector', {band, instrument, detector, physical_filter, visit}, SourceCatalog)]

In [ ]:
fps[:10]

In [ ]:
idx_fps = 0
the_fps = fps[idx_fps]
fps_centroid = the_fps.getCentroid()
fps_peaks = the_fps.getPeaks()
fps_npeaks = len(fps_peaks)
fps_area = the_fps.getArea()

In [ ]:
fps_peaks

In [ ]:
fps_centroid[0] 

In [ ]:
# We can get a rough view of the first source's footprint from its span
#fps[0].getSpans()
fps[idx_fps].isHeavy()

In [ ]:
# Next, we make all the footprints heavy at the same time
#  by operating on the footprint set
fpset.makeHeavy(img.getMaskedImage())

# This means we have to redefine fps:
hfps = fpset.getFootprints()

In [ ]:
%matplotlib widget
fig,ax = plt.subplots(1,1,figsize=(10,10))
im = ax.imshow(fps[idx_fps].getSpans().unflatten(hfps[idx_fps].getImageArray()),cmap='gray', origin='lower')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
the_title = f"footprint: fps_id = {idx_fps} ({fps_centroid[0]:.2f},{fps_centroid[1]:.2f}) , visit = {visitId}, band = {band}"
ax.set_title(the_title)
ax.grid()
ax.set_aspect('equal')
plt.colorbar(im, cax=cax)
plt.suptitle(collections)
plt.tight_layout()

In [ ]:
ra0 = 122.54952
dec0 = -36.16994

In [ ]:
ra0_deg = geom.Angle(ra0,geom.degrees)
dec0_deg = geom.Angle(dec0,geom.degrees)

In [ ]:
spt = geom.SpherePoint(longitude = ra0_deg,latitude= dec0_deg)

In [ ]:
pt = wcs.skyToPixel(spt)

In [ ]:
pt[0]

In [ ]:
xmin= pt[0]
ymin=pt[1]
width = 100
height= 100
extent = [xmin,xmin+width,ymin,ymin+height]

ramin,decmin = wcs.pixelToSky(x=float(xmin), y=float(ymin))
ramax,decmax = wcs.pixelToSky(x=float(xmin+width), y=float(ymin+height))

# Define a small region for a cutout
bbox = geom.Box2I()
bbox.include(geom.Point2I(xmin, ymin))
bbox.include(geom.Point2I(xmin + width, ymin + height))

# An alternative way to defined the same cutout region
# bbox = geom.Box2I(geom.Point2I(xmin, ymin), geom.Extent2I(width, height))

# Generate the cutout image
cutout = img.Factory(img, bbox, origin=afwImage.LOCAL, deep=False)

In [ ]:
ramin,decmin,ramax,decmax

In [ ]:
ra0 = (ramin+ramax)/2
dec0 = (decmin+decmax)/2

In [ ]:
%matplotlib widget
# Display the cutout and sources with afw display
image = cutout.image
fig,ax = plt.subplots(1,1,figsize=(8,8))
#im=ax.imshow(image.array, origin="lower",vmin=vmin, vmax=vmax, extent=extent ,cmap="gray")
im=ax.imshow(image.array, origin="lower", extent=extent ,cmap="gray")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
the_title = f"Calexp cutout visit = {visitId}, band = {band}"
ax.set_title(the_title)
ax.grid()
plt.colorbar(im, cax=cax)
plt.suptitle(collections)
plt.tight_layout()
plt.show()

## Many cutout on several visits

In [ ]:
nRow = 10
nCol = 5 
nIMG = nRow*nCol 

In [ ]:
spt

In [ ]:
fig, axes = plt.subplots(nRow, nCol, figsize=(3*nCol, 3*nRow), constrained_layout=True)
for idx, ax in enumerate(axes.flat):
    if idx >= nIMG:
        break
    visitId = my_selectedvisits[idx]
    dataId = {'visit': visitId, 'instrument':instrument_sel , 'detector': 0}

    # call butler
    exp = butler.get("calexp", dataId=dataId, collections=collections)
    wcs = exp.getWcs()
    bbox = exp.getBBox()

    # must test if the calexp include the selected point
    corners_ra, corners_dec = get_corners_radec(wcs, bbox)
    # 2D point
    pt = wcs.skyToPixel(spt)

    xmin= pt[0]
    ymin=pt[1]
    width = 100
    height= 100
    extent = [xmin,xmin+width,ymin,ymin+height]

    ramin,decmin = wcs.pixelToSky(x=float(xmin), y=float(ymin))
    ramax,decmax = wcs.pixelToSky(x=float(xmin+width), y=float(ymin+height))

    # Define a small region for a cutout
    bbox = geom.Box2I()
    bbox.include(geom.Point2I(xmin, ymin))
    bbox.include(geom.Point2I(xmin + width, ymin + height))

    # An alternative way to defined the same cutout region
    # bbox = geom.Box2I(geom.Point2I(xmin, ymin), geom.Extent2I(width, height))

    try:
    # Generate the cutout image
        cutout = exp.Factory(exp, bbox, origin=afwImage.LOCAL, deep=False)

        im=ax.imshow(cutout.image.array, origin="lower", extent=extent ,cmap="gray")
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        the_title = f"visit = {visitId}, f = {band}"
        ax.set_title(the_title)
        ax.grid()
        plt.colorbar(im, cax=cax,ax=ax)
    except Exception as inst:

        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)          # __str__ allows args to be printed directly,

        

plt.tight_layout()
    

In [ ]:
assert False

In [ ]:
# Plot cutouts centered on matched sources from different visits for a couple of objects.
# Selecting from the small/largeStdLists made above to look at the extremes.
imMin, imMax, Q = -0.001, 0.004, 8
expMin, expMax = -25, 100
numSub = 8
for maxBoxSize in [71]:
    plt.close("all")
    for objInd in smallStdList[0:3] + largeStdList[1:2]:
        joinedObjects = dataJoined[dataJoined["obj_index"] == objInd]
        numObj = len(joinedObjects)
        print("numObj = {} [with obj_index = {}]".format(numObj, objInd))
        nRow = 1 if numObj <= numSub else (numObj // numSub) + 1
        # print("nRow = {}".format(nRow))
        fig, axes = plt.subplots(nRow, numSub, figsize=(3*numSub, 3*nRow), constrained_layout=True)
        joinedObjects = dataJoined[dataJoined["obj_index"] == objInd]
        
        psfMag = joinedObjects["psfMag"]
        psfMagErr = joinedObjects["psfMagErr"]
        psfMagDiffMmag = joinedObjects["psfMagDiffMmag"].values
        psfMagStdMmag = psfMagStdMmagDict[objInd]

        clippedMeanRaArray = scipy.stats.sigmaclip(joinedObjects["ra"], low=3.0, high=3.0).clipped
        clippedMeanRa = np.mean(clippedMeanRaArray)
        clippedMeanDecArray = scipy.stats.sigmaclip(joinedObjects["dec"], low=3.0, high=3.0).clipped
        clippedMeanDec = np.mean(clippedMeanDecArray)

        iObj = -1
        for ax in axes.ravel():
            iObj += 1
            if iObj < numObj:
                row = joinedObjects[iObj:iObj + 1]
                detector = row["detector"].values[0]
                visit = row["visit"].values[0]
                psfMag = (row["psfFlux"].values[0]*u.nJy).to(u.ABmag)

                dataId = {"detector": detector, "visit": visit}
                exp = butler.get("calexp", dataId)
                wcs = exp.wcs
                pt = geom.SpherePoint(geom.Angle(row["ra"].values[0], geom.degrees),
                                      geom.Angle(row["dec"].values[0], geom.degrees))
                (xSrc, ySrc) = wcs.skyToPixel(pt)

                boxSize = int(min(maxBoxSize, xSrc, exp.getWidth()-xSrc, ySrc, exp.getHeight()-ySrc) - 1)

                minBbox = geom.Point2I(int(xSrc) - boxSize ,int(ySrc) - boxSize)
                maxBbox = geom.Point2I(int(xSrc) + boxSize, int(ySrc) + boxSize)
                srcBbox = geom.Box2I(minBbox, maxBbox)

                subimg = afwImage.ExposureF(exp, srcBbox, afwImage.PARENT, True)
                subimgRgb = afwRgb.makeRGB(subimg.image.array, None, None, expMin, expMax - expMin, Q)

                im = ax.imshow(subimgRgb, interpolation="nearest", origin="lower")
                ax.text(0.5, 0.06, "psfSig: {:.2f} pix".format(row["psfSigma"].values[0]),
                        color="cyan", fontsize=14, ha="center", va="center", transform=ax.transAxes)
                ax.text(0.5, 0.15, "{:.2f}".format(psfMag),
                        color="violet", fontsize=14, ha="center", va="center", transform=ax.transAxes)

                diffColor = "red" if np.abs(psfMagDiffMmag[iObj]) > 100 else "lawngreen"
                ax.text(0.5, 0.92, "diff: {:.2f} mmag".format(psfMagDiffMmag[iObj]),
                        color=diffColor, fontsize=14, ha="center", va="center", transform=ax.transAxes)

                ax.set_title("{} {}sec".format(visit, row["expTime"].values[0]), fontsize=12)

        color = "green"
        plt.text(0.5, 0.84, "std: {:.2f} mmag".format(psfMagStdMmag), color=color, fontsize=14,
                 ha="center", va="center", transform=ax.transAxes)

        if doSaveFigs:
            filename = "{}{}_allOjbectsSmallStd_{}_{}_{}".format(saveDir, camera.getName(), objInd, maxBoxSize, collectionStr)
            print("Saving file in: {}".format(filename))
            plt.savefig(filename, dpi=250)